In [26]:
import re
from cltk.corpus.greek.beta_to_unicode import Replacer
r = Replacer()

lemmahits = {}
lemmafails = {}

def searchalts(curkey):
    curlemma,startcit = curkey.split('-')
    hits = []
    citlist = []
    chars = list(startcit)
    m = re.search('^([0-9])\.',startcit)
    if(m):
        curbook = m[1]
        for i in range(1,9):
            newbook = str(i)
            if(curbook == newbook):
                continue
            newcit = re.sub('^([0-9])',newbook,startcit)
            if(newcit == startcit):
                continue
            citlist.append(newcit)
    
    curpat = '^([0-9]\.)([0-9])([0-9])'
    m = re.search(curpat,startcit)
    if(m):
        curunit = m[1]
        for i in range(1,10):
            newunit = str(i)
            if(curunit == newunit):
                continue
            newcit = re.sub(curpat,'\g<1>'+newunit+'\g<3>',startcit)
            if(newcit == startcit):
                continue
            citlist.append(newcit)
        
    
    curpat = '^([0-9]\.[0-9]|[0-9]\.)([0-9])(\.)'
    m = re.search(curpat,startcit)
    if(m):
        curunit = m[1]
        for i in range(0,10):
            newunit = str(i)
            if(curunit == newunit):
                continue
            newcit = re.sub(curpat,'\g<1>'+newunit+'\g<3>',startcit)
            if(newcit == startcit):
                continue
            citlist.append(newcit)
        
    curpat = '^([0-9]\.[0-9]+\.)([0-9])'
    m = re.search(curpat,startcit)
    if(m):
        curunit = m[1]
        for i in range(1,10):
            newunit = str(i)
            if(curunit == newunit):
                continue
            newcit = re.sub(curpat,'\g<1>'+newunit,startcit)
            if(newcit == startcit):
                continue
            citlist.append(newcit)
            
    for curcit in citlist:
        newkey = curlemma + '-' + curcit
        #print(newkey)
        if(newkey in glauxlemmaindex):
            hits.append(newkey)
            #print('hit!',newkey)
    return(hits)

def addentry(curentry,curdict):
    if(curentry in curdict):
        curdict[curentry] = curdict[curentry] + 1
    else:
        curdict[curentry] = 1

glauxlemmaindex = {}
glauxformindex = {}
glauxchapindex = {}

f = open('/Users/gcrane/github/glaux-trees/public/xml/0003-001.xml')

curcit = ''
curform = ''
curlemma = ''
for l in f:
    if(re.search('PUNCT',l)):
        continue
    m = re.search('subdoc="([^"]+)"',l)
    if(m):
        curcit = m[1]
        curbookchap = re.sub('\.[0-9]+$','',curcit)
       
    m = re.search('form="([^"]+)"',l)
    if(m):
        formcit = m[1] + '-' + curcit
        addentry(formcit,glauxformindex)
        
    m = re.search('lemma="([^"]+)"',l)
    if(m):
        formcit = m[1] + '-' + curcit
        addentry(formcit,glauxlemmaindex)
        lemmabookchap = m[1] + '-' + curbookchap
        if(not lemmabookchap in glauxchapindex):
            glauxchapindex[lemmabookchap] = []
        glauxchapindex[lemmabookchap].append(formcit)
            
        
    

f.close()


f = open('betant.thuclex_lateng.xml')
lemcits = 0
sawlemcit = 0

for l in f:
    m = re.search('lsj-([^"]+)',l)
    if(m):
        curlemma =  r.beta_code(m[1])
        lemmahits[curlemma] = []
        lemmafails[curlemma] = []
    cits = re.findall('<bibl n="[Tt]huc\.\s+([^"]+)"',l)
    for cit in cits:
        lemcit = curlemma + '-' + cit
        lemcits = lemcits + 1
        if(lemcit in glauxlemmaindex):
            sawlemcit = sawlemcit + 1
            lemmahits[curlemma].append(lemcit)
            #print('hit',lemcit)
        else:
            chapcit = re.sub('\.[0-9]+$','',lemcit)
            if(not chapcit in glauxchapindex):
                alts = searchalts(lemcit)
                if(not alts):
                    print('fail',lemcit)
                    lemmafails[curlemma].append(lemcit)
                elif(len(alts)>1):
                    print('ambig',lemcit,alts)
                else:
                    sawlemcit = sawlemcit + 1
                    lemmahits[curlemma].append(lemcit)
                
            else:
                sawlemcit = sawlemcit + 1
                lemmahits[curlemma].append(lemcit)
            #else:
                #print('chapcit',chapcit,glauxchapindex[chapcit])
            
    
f.close()

print(lemcits,sawlemcit,sawlemcit/lemcits)

fail ἀγαπάω-6.18.1
fail ἀγγέλλω-8.50.1
fail ἀγροῖκος-3.106.3
fail ἀγωγή-4.29.1
fail ἀγωγή-6.29.3
ambig ἀγών-7.59.2 ['ἀγών-7.49.2', 'ἀγών-7.69.2']
fail ἀδεής-8.89.2
ambig ἀεί-1.72.2 ['ἀεί-1.73.2', 'ἀεί-1.76.2']
fail ἀθυμέω-7.55.1
fail αἰτία-8.68.2
fail ἄκρα-1.46.4
fail ἄκρα-6.44.2
fail ἄκρα-7.4.4
fail ἄκρα-8.35.2
fail ἀκριβῶς-1.97.2
fail ἀκριβῶς-1.134.1
fail ἀκριβῶς-4.100.2
fail ἀκριβῶς-5.68.2
fail ἀκριβῶς-7.49.1
fail ἀκριβῶς-6.55.1
fail ἀκριβῶς-6.91.6
fail ἀκριβῶς-1.99.1
fail ἀκρόπολις-1.10.1
fail ἀκρόπολις-2.6.1
fail ἄκρος-4.100.2
fail ἄκρος-1.105.3
fail ἄκρος-4.25.9
fail ἄκρος-4.100.3
fail ἄκων-2.89.3
fail ἄκων-2.90.1
fail ἄκων-2.90.3
fail ἄκων-3.11.3
fail ἄκων-3.37.2
fail ἄκων-3.40.1
fail ἄκων-3.63.2
fail ἄκων-3.64.3
fail ἄκων-4.30.2
fail ἄκων-4.87.2
fail ἄκων-5.7.2
fail ἄκων-5.98.1
fail ἄκων-6.25.2
fail ἄκων-6.34.6
fail ἄκων-6.69.1
fail ἄκων-6.87.4
fail ἄκων-7.57.1
fail ἄκων-7.9.1
fail ἄκων-4.78.4
fail ἄκων-4.78.4
fail ἄκων-4.83.6
fail ἄκων-7.86.2
fail ἄκων-8.3.1
fail ἀλγεινός-7.68

fail ἄπειροσ-1.72.1
fail ἄπειροσ-1.81.6
fail ἄπειροσ-1.99.3
fail ἄπειροσ-1.138.3
fail ἄπειροσ-1.141.3
fail ἄπειροσ-2.4.2
fail ἄπειροσ-2.11.1
fail ἄπειροσ-2.35.2
fail ἄπειροσ-2.84.3
fail ἄπειροσ-3.53.4
fail ἄπειροσ-3.112.6
fail ἄπειροσ-4.29.4
fail ἄπειροσ-4.126.5
fail ἄπειροσ-6.1.1
fail ἄπειροσ-6.91.2
fail ἄπειροσ-7.61.2
fail ἄπειροσ-1.49.1
fail ἀπερίσκεπτος-2.13.5
fail ἀπέχθομαι-1.75.4
fail ἀπέχθομαι-1.76.1
fail ἀπέχθομαι-8.136.1
fail ἀπέχθομαι-2.63.1
fail ἀπιάλλω-5.77.8
fail ἄπειμι2,ἀπέρχομαι-1.24.5
fail ἄπειμι2,ἀπέρχομαι-1.26.5
fail ἄπειμι2,ἀπέρχομαι-1.45.1
fail ἄπειμι2,ἀπέρχομαι-1.89.3
fail ἄπειμι2,ἀπέρχομαι-1.92.1
fail ἄπειμι2,ἀπέρχομαι-1.95.7
fail ἄπειμι2,ἀπέρχομαι-1.105.7
fail ἄπειμι2,ἀπέρχομαι-1.108.2
fail ἄπειμι2,ἀπέρχομαι-1.126.8
fail ἄπειμι2,ἀπέρχομαι-1.134.1
fail ἄπειμι2,ἀπέρχομαι-2.34.6
fail ἄπειμι2,ἀπέρχομαι-2.52.4
fail ἄπειμι2,ἀπέρχομαι-2.82.1
fail ἄπειμι2,ἀπέρχομαι-2.101.5
fail ἄπειμι2,ἀπέρχομαι-3.4.4
fail ἄπειμι2,ἀπέρχομαι-3.18.1
fail ἄπειμι2,ἀπέρχομαι-3.62.5
fail ἄπειμ

fail δαιμόνιος-2.64.2
fail δάκρυον-1.121.3
fail δαίσ-7.53.4
fail δειλία-6.79.1
fail δεῖ-1.71.3
fail δεῖ-1.125.2
fail δεῖ-3.44.4
fail δεῖ-3.82.5
fail δεῖ-4.4.3
fail δεῖ-5.55.4
fail δεῖ-5.79.3
fail δεῖ-6.21.1
fail δεῖ-6.31.3
fail δεῖ-6.71.2
fail δεῖ-1.39.1
fail δεῖ-1.68.3
fail δεῖ-1.73.2
fail δεῖ-1.74.4
fail δεῖ-1.81.3
fail δεῖ-1.84.4
fail δεῖ-1.120.3
fail δεῖ-1.123.1
fail δεῖ-1.129.3
fail δεῖ-2.24.1
fail δεῖ-2.76.3
fail δεῖ-3.12.3
fail δεῖ-3.12.3
fail δεῖ-3.39.8
fail δεῖ-3.40.1
fail δεῖ-3.39.4
fail δεῖ-3.43.2
fail δεῖ-3.46.4
fail δεῖ-3.47.4
fail δεῖ-3.61.1
fail δεῖ-3.83.4
fail δεῖ-4.4.2
fail δεῖ-4.10.4
fail δεῖ-4.10.4
fail δεῖ-5.26.4
fail δεῖ-5.53.1
fail δεῖ-5.98.1
fail δεῖ-5.23.3
fail δεῖ-5.68.1
fail δεῖ-5.98.3
fail δεῖ-5.72.5
fail δεῖ-7.8.3
fail δεῖ-7.14.4
fail δεῖ-7.15.1
fail δεῖ-7.43.2
fail δεῖ-7.49.2
fail δεῖ-8.69.4
fail δεῖ-8.76.3
fail δεῖ-8.81.3
fail δεῖ-8.98.3
fail δεῖ-1.44.2
fail δεῖ-1.58.1
fail δεῖ-2.24.2
fail δεῖ-2.40.2
fail δεῖ-2.74.1
fail δεῖ-3.13.2
fail δεῖ-3.22.7
fail δεῖ

fail ἐκπλήσσω-4.36.2
fail ἐκπλήσσω-4.63.1
fail ἔκπλοος-7.50.3
fail ἔκπλοος-1.117.1
fail ἐκπολεμέω-8.57.1
fail ἐκπολεμόω-6.77.2
fail ἐκρήγνυμι-8.84.3
fail ἐκφέρω-4.12.1
fail ἐλάσσων-1.8.3
fail ἐλάσσων-1.11.1
fail ἐλάσσων-1.11.2
fail ἐλάσσων-1.40.6
fail ἐλάσσων-1.49.6
fail ἐλάσσων-2.11.4
fail ἐλάσσων-2.13.4
fail ἐλάσσων-2.13.5
fail ἐλάσσων-2.9.1
fail ἐλάσσων-2.49.5
fail ἐλάσσων-2.65.2
fail ἐλάσσων-2.94.1
fail ἐλάσσων-2.97.3
fail ἐλάσσων-3.39.3
fail ἐλάσσων-3.112.1
fail ἐλάσσων-4.120.2
fail ἐλάσσων-6.31.2
fail ἐλάσσων-7.15.1
fail ἐλάσσων-7.22.1
fail ἐλάσσων-7.23.1
fail ἐλάσσων-7.28.3
fail ἐλάσσων-7.71.7
fail ἐλάσσων-7.85.4
fail ἐλάσσων-8.87.5
fail ἐλάσσων-8.109.1
fail ἐλάσσων-1.42.3
fail ἐλάσσων-3.40.3
fail ἐλάσσων-3.45.1
fail ἐλάσσων-3.56.5
fail ἐλάσσων-4.66.3
fail ἐλάσσων-6.36.4
fail ἐλάσσων-7.28.3
fail ἐλάσσων-1.54.2
fail ἐλάσσων-1.63.3
fail ἐλάσσων-2.31.2
fail ἐλάσσων-2.31.2
fail ἐλάσσων-2.89.7
fail ἐλάσσων-3.17.3
fail ἐλάσσων-3.68.2
fail ἐλάσσων-3.75.5
fail ἐλάσσων-3.87.3
fail ἐλάσσω

fail ἐπιτελέω-1.108.3
fail ἐπιτελέω-7.2.4
fail ἐπιτίθημι-1.64.1
fail ἐπιφανής-8.87.4
fail ἐπιχειρέω-2.89.4
fail ἐπιχείρησις-5.4.1
fail ἐπιχειρητέον-1.118.2
fail ἐπιχειρητέον-2.3.3
fail ἐπιχθόνιος-1.41.1
fail ἐποφείλω-8.5.5
fail ἑπτά-2.5.1
fail ἑπτακόσιοι-6.96.3
fail Ἑρμῆσπλ-6.27.1
fail Ἑρμῆσπλ-6.28.1
fail Ἑρμῆσπλ-6.28.2
fail Ἑρμῆσπλ-6.53.1
fail Ἑρμῆσπλ-6.53.1
fail Ἑρμῆσπλ-6.2.1
fail Ἑρμῆσπλ-6.60.4
fail Ἑρμῆσπλ-6.61.1
fail ἐρωτάω-1.90.5
fail ἐρωτάω-3.4.1
fail ἐρωτάω-4.40.2
fail ἐρωτάω-7.10.1
fail εἰσάγω-4.6.1
fail εἰσβάλλω-2.79.6
fail εἴσειμι,εἰσέρχομαι-1.106.2
fail εἴσειμι,εἰσέρχομαι-1.134.1
fail εἴσειμι,εἰσέρχομαι-2.2.1
fail εἴσειμι,εἰσέρχομαι-2.2.3
fail εἴσειμι,εἰσέρχομαι-2.3.1
fail εἴσειμι,εἰσέρχομαι-2.4.3
fail εἴσειμι,εἰσέρχομαι-2.5.1
fail εἴσειμι,εἰσέρχομαι-2.13.2
fail εἴσειμι,εἰσέρχομαι-2.19.1
fail εἴσειμι,εἰσέρχομαι-2.54.5
fail εἴσειμι,εἰσέρχομαι-2.72.2
fail εἴσειμι,εἰσέρχομαι-2.73.1
fail εἴσειμι,εἰσέρχομαι-3.25.1
fail εἴσειμι,εἰσέρχομαι-3.28.2
fail εἴσειμι,εἰσέρχομαι-3.66.2
fai

...

fail παρίστημι-6.78.1
fail παρίστημι-4.133.1
fail παροχή-6.85.2
fail πᾶς-1.123.1
fail πᾶς-1.143.1
ambig πᾶς-6.62.1 ['πᾶς-7.62.1', 'πᾶς-6.22.1', 'πᾶς-6.32.1', 'πᾶς-6.60.1', 'πᾶς-6.65.1']
ambig πᾶς-2.84.3 ['πᾶς-7.84.3', 'πᾶς-2.64.3']
fail πᾶς-5.4.1
ambig πᾶς-5.47.3 ['πᾶς-5.77.3', 'πᾶς-5.46.3']
ambig πᾶς-5.47.4 ['πᾶς-2.47.4', 'πᾶς-5.46.4']
fail πᾶς-5.47.7
ambig πᾶς-7.7.1 ['πᾶς-8.7.1', 'πᾶς-7.6.1']
ambig πᾶς-8.4.1 ['πᾶς-8.2.1', 'πᾶς-8.7.1']
fail πᾶς-1.41.3
fail πάτριος-5.4.1
fail πεδίον-5.5.1
fail πεζικός-6.33.2
fail πεζικός-7.7.2
fail πεζικός-7.16.1
fail πεζός-4.10.1
fail πεζός-7.3.1
ambig πείθω-6.10.1 ['πείθω-6.50.1', 'πείθω-6.17.1']
fail πείθω-7.48.3
fail πείθω-4.68.6
fail πεῖρα-1.138.2
fail πειράω-4.25.10
fail πελταστής-4.2.1
ambig πέμπω-6.3.1 ['πέμπω-3.3.1', 'πέμπω-6.8.1']
fail πέμπω-6.10.1
ambig πέμπω-7.81.1 ['πέμπω-2.81.1', 'πέμπω-7.83.1']
ambig πέμπω-8.4.1 ['πέμπω-5.4.1', 'πέμπω-8.6.1']
ambig πέμπω-8.4.1 ['πέμπω-5.4.1', 'πέμπω-8.6.1']
ambig πέμπω-6.3.1 ['πέμπω-3.3.1', 'πέμπω-6.8.1'

fail πολύς,πλείων,πλεῖστος-2.29.2
fail πολύς,πλείων,πλεῖστος-3.37.3
fail πολύς,πλείων,πλεῖστος-3.82.7
fail πολύς,πλείων,πλεῖστος-4.117.2
fail πολύς,πλείων,πλεῖστος-2.44.4
fail πολύς,πλείων,πλεῖστος-4.117.1
fail πολύς,πλείων,πλεῖστος-1.77.3
fail πολύς,πλείων,πλεῖστος-3.71.1
fail πολύς,πλείων,πλεῖστος-4.17.4
fail πολύς,πλείων,πλεῖστος-4.21.2
fail πολύς,πλείων,πλεῖστος-4.92.2
fail πολύς,πλείων,πλεῖστος-1.12.4
fail πολύς,πλείων,πλεῖστος-1.73.5
fail πολύς,πλείων,πλεῖστος-1.118.2
fail πολύς,πλείων,πλεῖστος-2.98.4
fail πολύς,πλείων,πλεῖστος-3.11.1
fail πολύς,πλείων,πλεῖστος-3.35.2
fail πολύς,πλείων,πλεῖστος-3.36.5
fail πολύς,πλείων,πλεῖστος-3.108.1
fail πολύς,πλείων,πλεῖστος-3.108.3
fail πολύς,πλείων,πλεῖστος-4.4.3
fail πολύς,πλείων,πλεῖστος-4.106.1
fail πολύς,πλείων,πλεῖστος-5.114.2
fail πολύς,πλείων,πλεῖστος-7.2.4
fail πολύς,πλείων,πλεῖστος-7.57.10
fail πολύς,πλείων,πλεῖστος-7.11.1
fail πολύς,πλείων,πλεῖστος-8.48.3
fail πολύς,πλείων,πλεῖστος-2.100.6
fail πολύς,πλείων,πλεῖστος-4.86.4
fail πο

fail προίστημι-2.65.5
fail προίστημι-3.11.5
fail προίστημι-3.70.3
fail προίστημι-3.82.8
fail προίστημι-6.28.2
fail προίστημι-6.89.6
fail προίστημι-8.17.2
fail προίστημι-8.65.2
fail προίστημι-8.75.2
fail προίστημι-8.81.1
fail προίστημι-8.90.1
fail προίσχω-3.68.1
fail πρόκάθημαι-8.76.5
fail προκαλέω-7.3.1
fail προκαταλαμβάνω-4.2.1
fail προκατάρχομαι-1.25.4
fail προεῖπον-1.26.5
fail προεῖπον-1.43.1
fail προεῖπον-1.45.3
fail προεῖπον-1.45.4
fail προεῖπον-1.139.1
fail προεῖπον-2.8.4
fail προεῖπον-2.13.1
fail προεῖπον-3.66.1
fail προεῖπον-3.104.2
fail προεῖπον-4.9.2
fail προεῖπον-4.26.5
fail προεῖπον-4.80.3
fail προεῖπον-4.85.1
fail προεῖπον-4.128.1
fail προεῖπον-5.30.5
fail προεῖπον-5.64.3
fail προεῖπον-6.61.5
fail προεῖπον-6.65.1
fail προεῖπον-6.91.3
fail προεῖπον-7.50.3
fail προεῖπον-8.14.1
fail προεῖπον-1.118.1
fail προεῖπον-2.84.1
fail προεῖπον-5.31.1
fail προεῖπον-5.31.5
fail προεῖπον-8.69.3
fail προεῖπον-8.102.2
fail προεῖπον-4.130.5
fail προπάσχω-3.82.7
fail προπάσχω-6.38.4
fail προπ

fail στόρεννυμι-2.34.3
fail σφάλλω-3.84.3
fail σχεδία-3.68.4
fail σχεδία-5.66.4
fail σχεδία-7.33.2
fail σχολή-1.84.1
fail σχολή-2.75.6
fail σχολή-4.47.3
fail σχολή-7.15.2
fail σχολή-7.81.2
fail σώζω-1.63.1
fail σώζω-1.70.2
fail σώζω-1.74.1
fail σώζω-1.91.4
fail σώζω-1.129.3
fail σώζω-1.137.2
fail σώζω-2.40.4
fail σώζω-4.10.5
fail σώζω-4.23.2
fail σώζω-4.61.2
fail σώζω-5.69.2
fail σώζω-5.112.2
fail σώζω-6.9.3
fail σώζω-6.38.1
fail σώζω-6.40.2
fail σώζω-6.60.3
fail σώζω-6.78.3
fail σώζω-6.92.5
fail σώζω-8.76.6
fail σώζω-1.110.1
fail σώζω-1.121.5
fail σώζω-1.136.3
fail σώζω-2.4.2
fail σώζω-2.42.4
fail σώζω-2.63.3
fail σώζω-3.24.3
fail σώζω-3.64.3
fail σώζω-3.80.1
fail σώζω-3.112.8
fail σώζω-4.34.3
fail σώζω-4.62.2
fail σώζω-4.126.5
fail σώζω-5.27.2
fail σώζω-5.65.5
fail σώζω-5.73.3
fail σώζω-5.73.4
fail σώζω-5.91.2
fail σώζω-5.103.2
fail σώζω-5.111.2
fail σώζω-5.112.2
fail σώζω-6.24.3
fail σώζω-6.80.2
fail σώζω-6.84.1
fail σώζω-6.87.4
fail σώζω-7.44.8
fail σώζω-7.45.2
fail σώζω-7.56.2
fai

fail χείρων-1.95.7
fail χείρων-1.122.3
fail χείρων-1.138.3
fail χείρων-2.11.2
fail χείρων-2.42.3
fail χείρων-2.45.1
fail χείρων-2.45.2
fail χείρων-2.62.3
fail χείρων-3.9.1
fail χείρων-3.9.2
fail χείρων-3.37.3
fail χείρων-3.65.3
fail χείρων-4.114.3
fail χείρων-4.114.4
fail χείρων-5.111.4
fail χείρων-6.18.7
fail χείρων-6.89.3
fail χείρων-6.92.2
fail χείρων-7.71.1
fail χείρων-8.45.2
fail χείρων-8.76.6
fail χείρων-1.41.3
fail χείρων-1.73.1
fail χείρων-2.35.1
fail χείρων-2.43.1
fail χείρων-2.87.9
fail χείρων-3.46.1
fail χείρων-6.16.6
fail χείρων-6.80.3
fail χείρων-6.89.1
fail χείρων-6.89.6
fail χείρων-7.67.4
fail χείρων-8.87.3
fail χερσόνησος-1.11.1
fail χερσόνησος-8.62.2
fail χερσόνησος-8.99.1
fail χερσόνησος-8.102.1
fail χερσόνησος-8.104.2
fail χίλιοι-2.5.1
fail χίλιοι-2.5.1
fail χίλιοι-2.6.1
fail χίλιοι-2.7.1
fail χλωρός-2.5.1
fail χρῆμα-2.5.1
fail χρῆμα-2.6.1
fail χρῆμα-2.2.1
fail χρῆμα-6.4.1
fail χρῆμα-6.5.1
fail χρῆμα-7.5.1
fail χρῆμα-7.6.1
ambig χρῆμα-8.6.1 ['χρῆμα-8.3.1', 'χρῆμα-8.8

In [19]:
print('fails',lemmafails['ἀγαπάω'])
print('hits',lemmahits['ἀγαπάω'])

fails ['ἀγαπάω-6.18.1']
hits ['ἀγαπάω-6.36.4']


In [12]:
import re

glosses = {}

hilites = {}

glosstrans = {}

def addentry(curitem,curdict):
    if(curitem in curdict):
        curdict[curitem] = curdict[curitem] + 1
    else:
        curdict[curitem] = 1
        


def dofile1(fname):
    
    f = open(fname)
    #outfname = re.sub('la-unic\.xml','eng-unic2.xml',fname)
    #outf = open(outfname,'w')
    lnum = 0
    intext = 0
    for l in f:
        l = re.sub('\s+$','',l)
        #l = re.sub('<hi[^>]+>.+?</hi>',' ',l)
        hits = re.findall('<hi[^>]+>(.+?)</hi>',l)
        for foo in hits:
            addentry(foo,glosses)
            
dofile1('betant.thuclex_lateng3.xml')
            
outf = open('hilites2.txt','w')

for foo in sorted(glosses,key=glosses.get,reverse=True):
    print(foo)

outf.close()



vulgo
Ibid.
item
nonnulli codd.
similiter
Vat.
idem,
Ibid. bis.
praeterea vulgo
ubi nunc
Transl.
et
cf. Popp. adn.
vel
pro
ubi vulgo
alii
olim
ex Hom. H. Ap.
vid.
cum
item Ib.
boni codd.
ubi
Mosqu.
multi codd.
Ibid. ter.
ubi nonnulli codd.
depravati codd.
vulgo additur
Absolute,
om. nonnulli codd.
de pestilentia
plurimi codd.
vulgo pro
praeterea
Dor.
bis.
Bekk.
om.
de Sicilia
Vat. et alii
Neutr.
p.
alii leg.
ubi Vat.
de tempore,
pauci codd.
Schol.
de Alcibiade
de navibus,
Ib.
interpretantur
aut
adv.
Mosq.
de Brasida
ubi alii
plerique codd.
Impers.
Translate,
Neutr. subst.
Aor.
de hominibus,
de rebus,
ex. Hom. H. Ap.
de Pausania
cum infin.
Adverbialiter,
de Piraeo
Intrans.
i. e.
absolute,
in elogio
deest
de Themistocle
leg.
impers.
de navibus
in foedere Dorico
de loco,
c. acc.
Bekk. Goell.
ubi nunc deletum.
cf.
in multis codd.
Bas.
sub.
cum particip.
cum dat.
Vat. H.
codd.
sine artic.
nisi potius
de Pericle
malim
cum inf.
de Theseo
abest
absol.
de Homero
Reg.
cum accus.
cum acc. et inf.

In [14]:
print(len(glosses))
for foo in sorted(glosses,key=glosses.get,reverse=True):
    print(foo,glosses[foo])

1222
vulgo 768
Ibid. 408
item 248
nonnulli codd. 231
similiter 140
Vat. 114
idem, 93
Ibid. bis. 87
praeterea vulgo 82
ubi nunc 80
Transl. 66
et 64
cf. Popp. adn. 59
vel 56
pro 41
ubi vulgo 31
alii 31
olim 23
ex Hom. H. Ap. 22
vid. 21
cum 20
item Ib. 20
boni codd. 19
ubi 19
Mosqu. 18
multi codd. 16
Ibid. ter. 16
ubi nonnulli codd. 15
depravati codd. 15
vulgo additur 14
Absolute, 13
om. nonnulli codd. 12
de pestilentia 12
plurimi codd. 10
vulgo pro 10
praeterea 10
Dor. 10
bis. 10
Bekk. 9
om. 9
de Sicilia 9
Vat. et alii 9
Neutr. 9
p. 8
alii leg. 8
ubi Vat. 8
de tempore, 8
pauci codd. 8
Schol. 7
de Alcibiade 7
de navibus, 7
Ib. 7
interpretantur 7
aut 6
adv. 6
Mosq. 6
de Brasida 6
ubi alii 6
plerique codd. 6
Impers. 6
Translate, 6
Neutr. subst. 6
Aor. 6
de hominibus, 5
de rebus, 5
ex. Hom. H. Ap. 5
de Pausania 5
cum infin. 5
Adverbialiter, 5
de Piraeo 5
Intrans. 5
i. e. 5
absolute, 5
in elogio 5
deest 5
de Themistocle 5
leg. 5
impers. 5
de navibus 5
in foedere Dorico 5
de loco, 5
c. acc. 5


In [23]:
import re

glosses = {}

hilites = {}

glosstrans = {}

f = open('betlex-hilites.txt')

for l in f:
    l = re.sub('\s+$','',l)
    if(not re.search(' - ',l)):
        print('badline',l)
        continue
    args = l.split(' - ')
    glosstrans[args[0]] = l

f.close()

skips = ['Schol','FEM', 'NEUTR','SUBST','ACT','AOR','PASS','PART','DUAL','COMP','PLUR','MED','SUP','COMP','FUT','SING','I','II','Gr','PERF']

def addentry(curitem,curdict):
    if(curitem in curdict):
        curdict[curitem] = curdict[curitem] + 1
    else:
        curdict[curitem] = 1
        
def dofile(fname):
    
    f = open(fname)
    outfname = re.sub('lateng3.xml','lateng4.xml',fname)
    print(outfname)
    outf = open(outfname,'w')
    lnum = 0
    intext = 0
    for l in f:
        l = re.sub('\s+$','',l)
        savel = l
        lnum = lnum + 1
        if(re.search('<text',l)):
            intext = 1
            
        if(not intext):
            print(l,file=outf)
            continue
            
        hits = re.findall('<hi rend="italic">(.+?)</hi>',l)
        for hit in hits:
            if(hit in glosstrans):
                subs = re.sub(' - ','</hi> <hi rend="italic" xml:lang="eng">',glosstrans[hit])
                l = l.replace('<hi rend="italic">'+hit+'</hi>','<hi rend="italic" xml:lang="lat">'+subs+'</hi>')
        
        l = re.sub('(</hi>|,)(<b|<for|<quote)','\g<1> \g<2>',l)
        print(l,file=outf)
        
    
    f.close()
    outf.close()
    
    


    
dofile('betant.thuclex_lateng3.xml')
i = 0


betant.thuclex_lateng4.xml


In [20]:
glosstrans

{'vulgo': 'vulgo - commonly',
 'Ibid.': 'Ibid. - in the same place',
 'item': 'item - likewise',
 'nonnulli codd.': 'nonnulli codd. - several manuscripts',
 'similiter': 'similiter - similarly',
 'Vat.': 'Vat. - Vatican manuscript',
 'idem,': 'idem, - the same',
 'Ibid. bis.': 'Ibid. bis. - in the same place twice',
 'praeterea vulgo': 'praeterea vulgo - moreover in the common texts',
 'ubi nunc': 'ubi nunc - where now',
 'Transl.': 'Transl. - translate',
 'et': 'et - and',
 'cf. Popp. adn.': "cf. Popp. adn. - compare Poppo's note",
 'vel': 'vel - or',
 'pro': 'pro - for',
 'ubi vulgo': 'ubi vulgo - where commonly',
 'alii': 'alii - others',
 'olim': 'olim - formerly',
 'ex Hom. H. Ap.': "ex Hom. H. Ap. - from Homer's Hymn to Apollo",
 'vid.': 'vid. - see',
 'cum': 'cum - with',
 'item Ib.': 'item Ib. - likewise there',
 'boni codd.': 'boni codd. - good manuscripts',
 'ubi': 'ubi - where',
 'Mosqu.': 'Mosqu. - Moscow manuscript',
 'multi codd.': 'multi codd. - many manuscripts',
 'Ibid

In [31]:
import re

def searchalts(curkey):
    curlemma,startcit = curkey.split('-')
    print(curkey,curlemma,startcit)
    hits = []
    citlist = []
    chars = list(startcit)
    m = re.search('^([0-9])\.',startcit)
    if(m):
        curbook = m[1]
        for i in range(1,9):
            newbook = str(i)
            if(curbook == newbook):
                continue
            newcit = re.sub('^([0-9])',newbook,startcit)
            if(newcit == startcit):
                continue
            citlist.append(newcit)
    
    curpat = '^([0-9]\.)([0-9])([0-9])'
    m = re.search(curpat,startcit)
    if(m):
        curunit = m[1]
        for i in range(1,10):
            newunit = str(i)
            if(curunit == newunit):
                continue
            newcit = re.sub(curpat,'\g<1>'+newunit+'\g<3>',startcit)
            if(newcit == startcit):
                continue
            citlist.append(newcit)
        
    
    curpat = '^([0-9]\.[0-9]|[0-9]\.)([0-9])(\.)'
    m = re.search(curpat,startcit)
    if(m):
        curunit = m[1]
        for i in range(0,10):
            newunit = str(i)
            if(curunit == newunit):
                continue
            newcit = re.sub(curpat,'\g<1>'+newunit+'\g<3>',startcit)
            if(newcit == startcit):
                continue
            citlist.append(newcit)
        
    curpat = '^([0-9]\.[0-9]+\.)([0-9])'
    print('\n'+curpat)
    m = re.search(curpat,startcit)
    if(m):
        curunit = m[1]
        for i in range(1,10):
            newunit = str(i)
            if(curunit == newunit):
                continue
            newcit = re.sub(curpat,'\g<1>'+newunit,startcit)
            if(newcit == startcit):
                continue
            citlist.append(newcit)
            
    print(citlist)
    for curcit in citlist:
        newkey = curlemma + '-' + curcit
        print(newkey)
        if(newkey in glauxlemmaindex):
            print('hit!',newkey)
        
        
searchalts('πείθω-3.94.1')

πείθω-3.94.1 πείθω 3.94.1

^([0-9]\.[0-9]+\.)([0-9])
['1.94.1', '2.94.1', '4.94.1', '5.94.1', '6.94.1', '7.94.1', '8.94.1', '3.14.1', '3.24.1', '3.34.1', '3.44.1', '3.54.1', '3.64.1', '3.74.1', '3.84.1', '3.90.1', '3.91.1', '3.92.1', '3.93.1', '3.95.1', '3.96.1', '3.97.1', '3.98.1', '3.99.1', '3.94.2', '3.94.3', '3.94.4', '3.94.5', '3.94.6', '3.94.7', '3.94.8', '3.94.9']
πείθω-1.94.1
πείθω-2.94.1
πείθω-4.94.1
πείθω-5.94.1
πείθω-6.94.1
πείθω-7.94.1
πείθω-8.94.1
πείθω-3.14.1
hit! πείθω-3.14.1
πείθω-3.24.1
πείθω-3.34.1
πείθω-3.44.1
πείθω-3.54.1
hit! πείθω-3.54.1
πείθω-3.64.1
πείθω-3.74.1
πείθω-3.84.1
πείθω-3.90.1
πείθω-3.91.1
πείθω-3.92.1
πείθω-3.93.1
πείθω-3.95.1
hit! πείθω-3.95.1
πείθω-3.96.1
πείθω-3.97.1
πείθω-3.98.1
πείθω-3.99.1
πείθω-3.94.2
πείθω-3.94.3
πείθω-3.94.4
πείθω-3.94.5
πείθω-3.94.6
πείθω-3.94.7
πείθω-3.94.8
πείθω-3.94.9
